In [1]:
import numpy as np
from gsmmutils import ModelValidator, MyModel
from cobra.flux_analysis import pfba, flux_variability_analysis as fva
import gsmmutils
gsmmutils.DATA_PATH = "../data"
from cobra.io import to_json, save_json_model
from utils import *
import logging
from mewpy.problems import MediumProblem
from mewpy.optimization import EA
logging.getLogger('cobra').setLevel(logging.CRITICAL)
logging.getLogger('gsmmutils').setLevel(logging.CRITICAL)
logging.getLogger('mewpy').setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [2]:
model = MyModel(r"../data/model_no_carotenoids.xml", "e_Biomass__cytop")
for demand in model.demands:
    if demand.id not in ["DM_C02094__chlo", "DM_C08601__chlo"] and 'photon' not in demand.id:
        demand.bounds = (0, 0)
model.reactions.e_Biomass__cytop.bounds = (0.14, 0.25)
model.exchanges.EX_C00011__dra.bounds = (-8.61, 1000)
model.reactions.e_Pigment__chlo.bounds = (0,0)
constraint = model.problem.Constraint(
            model.reactions.R09503_hn438__lum.flux_expression + model.reactions.R09503_hn673__lum.flux_expression,
            lb=0,
            ub=199.44)
model.add_cons_vars(constraint)
model.summary()

Loading

Reactions: 4623
Metabolites: 3737
Genes: 1690
Model loaded
Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,7.08,0,0.00%
C00009__extr,EX_C00009__dra,0.0332,0,0.00%
C00011__extr,EX_C00011__dra,8.61,1,99.75%
C00059__extr,EX_C00059__dra,0.02207,0,0.00%
C00080__extr,EX_C00080__dra,0.05599,0,0.00%
C00120__extr,EX_C00120__dra,0.0008005,10,0.09%
C00205__extr,EX_C00205__dra,618.7,0,0.00%
C00244__extr,EX_C00244__dra,0.5456,0,0.00%
C00305__extr,EX_C00305__dra,0.0001543,0,0.00%
C00378__extr,EX_C00378__dra,0.0004601,12,0.06%


In [3]:
# Define the target

pfba_sol = pfba(model)
as_dict = pfba_sol.fluxes.to_dict()


PRODUCT_ID = 'DM_C02094__chlo'
BIOMASS_ID = 'e_Biomass__cytop'

# Optimization objectives
from mewpy.optimization.evaluation import  BPCY, WYIELD, BPCY_FVA
evaluator_1 = BPCY(BIOMASS_ID, PRODUCT_ID, method='pFBA')
evaluator_2 = WYIELD(BIOMASS_ID, PRODUCT_ID)
# evaluator = BPCY_FVA(BIOMASS_ID, PRODUCT_ID, method='lMOMA', reference =as_dict)

In [4]:
problem = MediumProblem(model, fevaluation=[evaluator_1, evaluator_2], reference=as_dict,
                        target = ["EX_C00244__dra", "EX_C00014__dra", "EX_C00086__dra",
                                  "EX_C00009__dra", 
                                  "EX_C00059__dra", "EX_C00320__dra",
                                  "EX_C00288__dra", "EX_C00033__dra", "EX_C00011__dra", "EX_C00267__dra", "EX_C00221__dra"],
                        # [ex.id for ex in model.exchanges if ex.id not in ["EX_C00001__dra",
                        #                                                           "EX_C00080__dra",
                        #                                                           "EX_C00011__dra"]],
                       levels = [0, 0.1, 0.5, 1, 3, 7, 10]
)

In [5]:
ea = EA(problem, max_generations=100)
final_pop = ea.run()
df = ea.dataframe()
df.to_csv("medium_opt_bpcy_wyield_filtered.csv")

2024-03-26 16:28:11,042	INFO worker.py:1752 -- Started a local Ray instance.


Using 32 workers.
Running NSGAII
Eval(s)|     Worst      Best    Median   Average   Std Dev|     Worst      Best    Median   Average   Std Dev|
    100|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000000  0.099541  0.000000  0.006859  0.020477|
    200|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000000  0.099541  0.000000  0.014003  0.026718|
    300|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.000675  0.099634  0.017999  0.039824  0.035301|
    400|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.043427  0.099634  0.081316  0.082971  0.014603|
    500|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.081316  0.099634  0.098672  0.096779  0.003674|
    600|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.098672  0.099634  0.099521  0.099329  0.000315|
    700|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.099541  0.099634  0.099634  0.099627  0.000024|
    800|  0.000000  0.000000  0.000000  0.000000  0.000000|  0.099634  0.099634

In [1]:
final_pop[1]

NameError: name 'final_pop' is not defined

In [7]:
from mewpy.simulation import get_simulator
simul = get_simulator(model)

In [25]:
solution = simul.simulate(method='pFBA', constraints=final_pop[1].constraints)
df = solution.dataframe
df.loc[df.index.str.startswith("DM_")]

ValueError: The model already has a pFBA objective.

In [23]:
fva = simul.FVA(constraints=final_pop[2].constraints, obj_frac=1.0)
fva["DM_C02094__chlo"]

[0.0, 0.3811239507692182]

In [11]:
constraints = {}
constraints['DM_C08601__chlo'] = (0, 0)
constraints['DM_C05306__chlo'] = (0, 0)
constraints['DM_C05307__chlo'] = (0, 0)
constraints['EX_C00533__dra'] = (0, 0)
constraints['DM_C18022__vacu'] = (0, 0)
constraints['EX_C00244__dra'] = (-0.5, 0)
constraints['EX_C00011__dra'] = (-10, -10)

In [12]:
for key, constr in constraints.items():
    model.reactions.get_by_id(key).bounds = constr

In [13]:
model.summary(fva=1.0)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
C00001__extr,EX_C00001__dra,7.568,[-4992; 28.55],0,0.00%
C00009__extr,EX_C00009__dra,0.03042,[0.03042; 0.03042],0,0.00%
C00011__extr,EX_C00011__dra,10,[10; 10],1,99.81%
C00059__extr,EX_C00059__dra,0.02022,[0.02022; 0.02022],0,0.00%
C00120__extr,EX_C00120__dra,0.0007336,[0.0007336; 0.0007336],10,0.07%
C00205__extr,EX_C00205__dra,661,[653.6; 855],0,0.00%
C00244__extr,EX_C00244__dra,0.5,[0.5; 0.5],0,0.00%
C00305__extr,EX_C00305__dra,0.0001414,[0.0001414; 0.0001414],0,0.00%
C00378__extr,EX_C00378__dra,0.0004216,[0.0004216; 0.0004216],12,0.05%
C02823__extr,EX_C02823__dra,0.000113,[0.000113; 0.000113],63,0.07%
